In [1]:
import pandas as pd
from pytube import YouTube
import pydub
import os
import re
import glob

os.environ["IMAGEIO_FFMPEG_EXE"] = "/opt/homebrew/bin/ffmpeg"

import moviepy.editor as mp


In [2]:
data = pd.read_csv("url.csv")

In [3]:
def download_video(tag, path):
    url = "https://www.youtube.com/watch?v=" + tag
    yt = YouTube(url, use_oauth=True, allow_oauth_cache=True)
    yt.streams.filter(file_extension='mp4').first().download(path)

    video = yt.streams.filter(only_audio=True).first()
    out_file = video.download(output_path=path)


def rename_file(name):
    os.rename("./data/mp4/" + name + ".mp4", "./data/mp4/" + str(i) + ".mp4")


def mp4_to_mp3(in_path, out_path):
    audio = mp.AudioFileClip(in_path)
    audio.write_audiofile(out_path)


def cut_descriptrion(in_path, out_path):
    mp3 = pydub.AudioSegment.from_mp3(in_path)

    slice = pydub.silence.split_on_silence(mp3, silence_thresh = -38, min_silence_len = 800, keep_silence = 1000)

    for i in range(len(slice)):
        if slice[0].duration_seconds <= 15 and slice[0].duration_seconds + slice[i + 1].duration_seconds < 60:
            slice[0] = slice[0].append(slice[i + 1])

    if slice[0].duration_seconds <= 15 or 60 <= slice[0].duration_seconds:
        raise Exception("cut fail")

    slice[0].export(out_path, format="mp3")

In [4]:
traincsv = open("./data/train.csv", "w")
traincsv.write("\"file\",\"Description\"\n")
failfile = open("./data/fail.txt", "w")

for i in range(len(data)):
    tag = data["ID"][i]
    video_name = re.sub(r'[.\\/*?:"<>|%]', "", data["Title"][i])
    mp4_path = "./data/mp4/" + str(i) + ".mp4"
    mp3_path = "./data/mp3/" + str(i) + ".mp3"
    description_path = "./data/description/" + str(i) + ".mp3"
    
    try:
        download_video(tag, "./data/mp4")
    except:
        print(i, data["Title"][i], "cant downlaod")
        failfile.write(str(i) + ". " + data["Title"][i] + " cant downlaod.\n")
        continue

    try:
        rename_file(video_name)
    except:
        print(i, data["Title"][i], "cant rename")
        failfile.write(str(i) + ". " + data["Title"][i] + " cant rename.\n")
        continue

    try:
        mp4_to_mp3(mp4_path, mp3_path)
    except:
        print(i, data["Title"][i], "cant convert")
        failfile.write(str(i) + ". " + data["Title"][i] + " cant convert.\n")
        continue

    try:
        cut_descriptrion(mp3_path, description_path)
    except:
        print(i, data["Title"][i], "cant cut")
        failfile.write(str(i) + ". " + data["Title"][i] + " cant cut.\n")
        continue

    try:
        description = re.sub(r'[「」『』()《》.\\/*:"<>|%#\\s]', "", data["Description"][i])
        traincsv.write("\"" + str(i) + ".mp3\",\"" + description + "\"\n")
    except:
        print(i, data["Title"][i], "cant write train.txt")
        failfile.write(str(i) + ". " + data["Title"][i] + " cant write train.txt.\n")
        continue

    print(i, data["Title"][i], "success")

traincsv.close()
failfile.close()
    



MoviePy - Writing audio in ./data/mp3/0.mp3


MoviePy - Done.
0 石虎生存危機 光電.生態.國土規畫如何兼顧【客家新聞20230719】 success
1 彩虹之國南非吸引移民 經濟蕭條.危機四伏【客家新聞20230719】 cant downlaod
MoviePy - Writing audio in ./data/mp3/2.mp3


MoviePy - Done.
